In [1]:
import tensorflow as tf

In [3]:
tf.__version__

'2.10.0'

In [14]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2594818199735481624
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2234830030
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15769931346922643075
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

## 베이직 스타일

### Define and Run 스타일

- 두 부분으로 나누어 작성
	1. Define
		- **데이터가 흘러가는 플로우를 구성**하는 파트
		- 연산의 입력, 출력까지의 흐름을 설계하는 것(인공신경망 구성)
		- 학습전에 이미, 최적화, 평가, 등등 플로우도 구성
		- 파이썬으로 구성, API는 텐서플로우 사용
		- 연산은 없음

	2. Run
		- **실제 데이터 주입/학습 -> 평가/최적화 -> 예측**
		- 세션이 실제적으로 처리
		- python --> cypthon --> C++ 연동
		- 단계
			1. ~~세션 오픈~~(2.0에선 삭제)
			2. 데이터 주입
			3. C++ 연산 수행 -> GPU를 이용하여 연산도 가능
			4. 연산이 끝나면 파이썬에게 전달
			5. 결과 확인
			6. ~~세션 닫기~~(2.0에선 삭제)

##### Tensorflow 2에서 1을 사용하고자 하면

```python
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

AttributeError: module 'tensorflow.compat.v1' has no attribute 'contrib'
```
결국엔 Tensorflow 1을 다시 받아야 함. 쓰지 말 것!

In [ ]:
### 실행하지 말것!
in_a = tf.placeholder(dtype=tf.float32, shape=(2))
in_b = tf.placeholder(dtype=tf.float32, shape=(2))

def forward(x):
  with tf.variable_scope("matmul", reuse=tf.AUTO_REUSE):
    W = tf.get_variable("W", initializer=tf.ones(shape=(2,2)),
                        regularizer=tf.contrib.layers.l2_regularizer(0.04))
    b = tf.get_variable("b", initializer=tf.zeros(shape=(2)))
    return W * x + b

out_a = forward(in_a)
out_b = forward(in_b)

reg_loss = tf.losses.get_regularization_loss(scope="matmul")

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  outs = sess.run([out_a, out_b, reg_loss],
                feed_dict={in_a: [1, 0], in_b: [0, 1]})

#### 다시 Tensorflow 2!

In [13]:
import tensorflow as tf
tf.__version__

'2.10.0'

##### 기본

In [21]:
# 랜덤 수 생성 - 균일분포
rand = tf.random.uniform([1], 0, 1)
print(rand)

Tensor("random_uniform_6/RandomUniform:0", shape=(1,), dtype=float32)


In [26]:
# 랜덤 여러수 생성 - 균일분포
rand = tf.random.uniform([4], 0, 1)
print(rand)

Tensor("random_uniform_11/RandomUniform:0", shape=(4,), dtype=float32)


In [29]:
# 랜덤 여러수 생성 - 정규분포
rand = tf.random.normal([4], 0, 1)
print(rand)

Tensor("random_normal:0", shape=(4,), dtype=float32)


##### 뉴런 생성

In [36]:
import math

In [37]:
# 시그모이드 함수
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [38]:
## 뉴런 입출력 정의
x, y = 1, 0
w = tf.random.normal([1],0,1)
sigmoid(x * w)

TypeError: must be real number, not Tensor